In [5]:
import os
import re
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import torch
from transformers import AutoModel, AutoTokenizer
import gradio as gr
import mdtex2html
import platform
from transformers import AutoTokenizer, AutoModel
from utility.utils import config_dict
from utility.loggers import logger
from sentence_transformers import util
from local_database import db_operate
from utils import obtain_sql, retrieval_related_table, execute_sql
from prompt import query_template, chatbot_prompt




In [2]:

tokenizer = AutoTokenizer.from_pretrained("/data1/dxw_data/llm/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/data1/dxw_data/llm/chatglm3-6b", trust_remote_code=True).half().cuda()
model = model.eval()



Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.18s/it]


In [3]:

def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    print(text)
    return text

def predict(input, history):
    max_length = 2048
    top_p = 0.7
    temperature = 0.2
    dboperate = db_operate(config_dict['db_path'])
    input_prompt = chatbot_prompt
    input_prompt = retrieval_related_table(input_prompt, input, history, top_k=3)
    input_prompt += query_template
    query = input_prompt.replace("<user_input>", input)
    
    response, history = model.chat(tokenizer, query, history=history, max_length=max_length, top_p=top_p, temperature=temperature)
    
    response = parse_text(response)
    response = obtain_sql(response)
    
    chatbot = [("", "")]  # 初始化 chatbot 列表并添加一个空条目
    print(f"Debug: Initial chatbot: {chatbot}")  # 打印初始 chatbot 列表
    chatbot = execute_sql(response, chatbot, dboperate)
    
    return response, history



In [4]:

# 模拟用户输入和历史记录
user_input = "请帮我查询所有的事件类型"
history = []

# 调用predict函数
response, history = predict(user_input, history)

# 打印response
print(response)

SELECT event_type FROM event_record
Debug: Initial chatbot: [('', '')]
Debug: Response in execute_sql: SELECT event_type FROM event_record
Debug: Chatbot before SQL execution: [('', '')]
Debug: Chatbot before updating: [('', '')]
Debug: Chatbot after updating: [('', "\n\n====================\n\nsql语句执行成功,结果如下:\n\n[('打架',), ('聚众',), ('翻越',)]")]
SELECT event_type FROM event_record
